<a href="https://colab.research.google.com/github/mikedcurry/DS-Unit-1-Sprint-3-Statistical-Tests-and-Experiments/blob/master/Curry_LS_DS_131_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 3 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

## Pre-Watch Vid Notes

In [0]:
from scipy import stats
#dir(stats) #directory of all that's in stats

In [0]:
norm = stats.norm()
print(norm.mean())
print(norm.std())
print(norm.var())

0.0
1.0
1.0


In [0]:
tl = stats.t(5) 
print(tl.mean())
print(tl.std())
print(tl.var())

0.0
1.2909944487358056
1.6666666666666667


## Cleaning

In [0]:
import pandas as pd

In [0]:
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data', header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y


In [0]:
# We got some cleaning to do

From: https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records

Attribute Information:

1. Class Name: 2 (democrat, republican) 
2. handicapped-infants: 2 (y,n) 
3. water-project-cost-sharing: 2 (y,n) 
4. adoption-of-the-budget-resolution: 2 (y,n) 
5. physician-fee-freeze: 2 (y,n) 
6. el-salvador-aid: 2 (y,n) 
7. religious-groups-in-schools: 2 (y,n) 
8. anti-satellite-test-ban: 2 (y,n) 
9. aid-to-nicaraguan-contras: 2 (y,n) 
10. mx-missile: 2 (y,n) 
11. immigration: 2 (y,n) 
12. synfuels-corporation-cutback: 2 (y,n) 
13. education-spending: 2 (y,n) 
14. superfund-right-to-sue: 2 (y,n) 
15. crime: 2 (y,n) 
16. duty-free-exports: 2 (y,n) 
17. export-administration-act-south-africa: 2 (y,n)

In [0]:
# Looks like I'm missing a header

# added the ", header=None" to my import of the csv

df.columns = ['party', 'handicapped-infants', 'water-project-cost-sharing', 'adoption-of-the-budget-resolution', 
              'physician-fee-freeze', 'el-salvador-aid', 'religious-groups-in-schools', 'anti-satellite-test-ban', 
             'aid-to-nicaraguan-contras', 'mx-missile', 'immigration', 'synfuels-corporation-cutback', 'education-spending', 
              'superfund-right-to-sue', 'crime', 'duty-free-exports', 'export-administration-act-south-africa']
df.head(2)


,party,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN


In [0]:
import numpy as np

In [0]:
# Time to replace y with 1, n with 0, and ? as NaN

df = df.replace({'y': 1, 'n':0, '?': np.nan})
df.head()

#Cleaned. Time for "lunch".

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


In [0]:
# T-test functions import

from scipy.stats import ttest_ind, ttest_ind_from_stats, ttest_rel


##The Five Parts of a T-test EXAMPLE from Lecture

**Before:**

1) Null Hypothesis: "The Boring Hypothesis", "The default state of the world"

 - The cooking times between the two burners is the same: $\mu_{1} = \mu_{2}$

2) Alternative Hypothesis: "The opposite of the null hypothesis"

- The cooking times are different $\mu_{1} \neq \mu_{2}$

3) Confidence Level: 95%

**Confidence level of 95%** (reach an incorrect conclusion only 5% of the time)

p < 1 - Confidence Level

1 - Confidence Level = $\alpha$

- I reject the null hypothesis when P-value < alpha (1 - Confidence Level)


 **After:**

4) T-statistic: 6.362

5) P-value: .000000000251

What is the t-statistic?: ~Roughly the number of standard deviations away from the mean, that corresponds to the differences in means that we have observed given their sample sizes. 

What is the P-value: The probability that the difference that we have observed (The t-statist that we get) could have happened by random chance. 

### Interpretation: 

Due to calculating a t-statistic of 6.362 which corresponds to a p-value of .000000000251, we reject the null hypothesis that the mean of cooking times between the two burners is equal, and suggest the alternative hypothesis that they are different.

## Example of T-test on this df from lecture

In [0]:
from scipy.stats import ttest_1samp

In [0]:
# Make two dfs

rep = df[df['party'] == 'republican']
dem = df[df['party'] == 'democrat']

dem.head()
#rep.head()

,party,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0
5,democrat,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
6,democrat,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0,1.0,1.0


In [0]:
# how dems voted on this issue
# Still just copying out the example in lecture here 

dem['handicapped-infants'].value_counts(dropna=False) # like this dropna=False thing

1.0    156
0.0    102
NaN      9
Name: handicapped-infants, dtype: int64

In [0]:
dem['handicapped-infants'].mean()

0.6046511627906976

In [0]:
# 1 sample t-test

ttest_1samp(dem['handicapped-infants'], .5, nan_policy='omit')

# Looks like we are in a pretty high confidence interval if p-value is so low

Ttest_1sampResult(statistic=3.431373087696574, pvalue=0.000699612317167372)

In [0]:
# Now running the same test on the Reps, not pretty, but ALL AT ONCE...

print(rep['handicapped-infants'].value_counts(dropna=False))
print('mean: ', rep['handicapped-infants'].mean())
print(ttest_1samp(dem['handicapped-infants'], .5, nan_policy='omit'))


0.0    134
1.0     31
NaN      3
Name: handicapped-infants, dtype: int64
mean:  0.18787878787878787
Ttest_1sampResult(statistic=3.431373087696574, pvalue=0.000699612317167372)


In [0]:
# Now running the 2 sample test...

ttest_ind(dem['handicapped-infants'], rep['handicapped-infants'], nan_policy='omit')

Ttest_indResult(statistic=9.205264294809222, pvalue=1.613440327937243e-18)

## water-project-cost-sharing:


In [0]:
dem['water-project-cost-sharing'].mean()

0.502092050209205

In [0]:
rep['water-project-cost-sharing'].mean()

0.5067567567567568

In [0]:
dem['water-project-cost-sharing'].value_counts(dropna=False)

1.0    120
0.0    119
NaN     28
Name: water-project-cost-sharing, dtype: int64

In [0]:
rep['water-project-cost-sharing'].value_counts(dropna=False)

1.0    75
0.0    73
NaN    20
Name: water-project-cost-sharing, dtype: int64

In [0]:
# Intesting

In [0]:

  
print(ttest_1samp(dem['water-project-cost-sharing'], .5, nan_policy='omit'))
print(ttest_1samp(rep['water-project-cost-sharing'], .5, nan_policy='omit'))

ttest_ind(dem['water-project-cost-sharing'], rep['water-project-cost-sharing'], nan_policy='omit')

Ttest_1sampResult(statistic=0.06454972243678961, pvalue=0.9485867005339235)
Ttest_1sampResult(statistic=0.16385760607458383, pvalue=0.8700683158522193)


Ttest_indResult(statistic=-0.08896538137868286, pvalue=0.9291556823993485)

In [0]:
df['water-project-cost-sharing']



After:

4) **T-statistic:** -0.089 -- Pretty much dead center

5) **P-value:** .929 

**Interpretation:**
Due to calculating a t-statistic of very very low and the P-value very very high, it is extremely likely that this is a non-partisan dicision. In fact both paties are pretty much split within themselves to a reasonably certain degree.

**NON-PARTISAN**

## adoption-of-the-budget-resolution

In [0]:
dem['adoption-of-the-budget-resolution'].mean()

0.8884615384615384

In [0]:
rep['adoption-of-the-budget-resolution'].mean()

0.13414634146341464

In [0]:
# This is already looking PRETTY partisan. Let's find out to what degree...

In [0]:
print(ttest_1samp(dem['handicapped-infants'], .5, nan_policy='omit'))

Ttest_1sampResult(statistic=3.431373087696574, pvalue=0.000699612317167372)


In [0]:
# First I want to make sure there isn't a crazy amount of obstaining



In [0]:
dem['adoption-of-the-budget-resolution'].value_counts(dropna=False)

1.0    231
0.0     29
NaN      7
Name: adoption-of-the-budget-resolution, dtype: int64

In [0]:
rep['adoption-of-the-budget-resolution'].value_counts(dropna=False)

0.0    142
1.0     22
NaN      4
Name: adoption-of-the-budget-resolution, dtype: int64

In [0]:
# And now the test:

ttest_ind(dem['adoption-of-the-budget-resolution'], rep['adoption-of-the-budget-resolution'], nan_policy='omit')

Ttest_indResult(statistic=23.21277691701378, pvalue=2.0703402795404463e-77)

In [0]:
# Yeah. So, about that. 

4) **T-statistic:** 23.2 !!!!

5) **P-value:** .000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000002

**Interpretation:**
Quite extreme results. Chances of the this being a coicidence are ***SLIM. ***

**THIS IS A DEMOCRAT ISSUE**

## physician-fee-freeze

In [0]:
dem['physician-fee-freeze'].mean()

0.05405405405405406

In [0]:
rep['physician-fee-freeze'].mean()

0.9878787878787879

In [0]:
#YEAH. pretty clear what's going on here already, but here we go...

In [0]:
dem['physician-fee-freeze'].value_counts(dropna=False)

0.0    245
1.0     14
NaN      8
Name: physician-fee-freeze, dtype: int64

In [0]:
rep['physician-fee-freeze'].value_counts(dropna=False)

1.0    163
NaN      3
0.0      2
Name: physician-fee-freeze, dtype: int64

In [0]:
ttest_ind(dem['physician-fee-freeze'], rep['physician-fee-freeze'], nan_policy='omit')

Ttest_indResult(statistic=-49.36708157301406, pvalue=1.994262314074344e-177)

4) T-statistic: -49.4 !!!!

5) P-value: Basically Zero

Interpretation: Quite extreme results. Chances of the this being a coicidence are *SLIM. *

**THIS IS A REPUBLICAN ISSUE**